<a href="https://colab.research.google.com/github/yagamiAbhi/Automobile-Manufacturing-Data-Analysis-pySpark/blob/main/Automobile_manufacturing_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider a leading two-wheeler manufacturing unit that produces two-wheeler components for the designs provided by various automobile companies

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=385d27457ed86b62b091ee92df786b8d3122d72959971b6032f0379027c82606
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
sc = SparkContext(master='local',appName='AMS')
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Schema for the Datasets

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType

In [ ]:
order_schema = StructType([StructField('OrderID', StringType(), True),
                           StructField('CompanyID', StringType(), True),
                           StructField('partsRequired', StringType(), True),
                           StructField('OrderQuantity', StringType(), True),
                           StructField('CostOfPart', StringType(), True),
                           StructField('OrderDate', StringType(), True),
                           StructField('ExpectedDeliveryDate', StringType(), True),
                           StructField('DeliveryStatus', StringType(), True)])

company_schema = StructType([StructField('CompanyID', StringType(), True),
                             StructField('CompanyName', StringType(), True),
                             StructField('CompanyLocation', StringType(), True),
                             StructField('CompanyContact', StringType(), True),
                             StructField('EstablishedYear', StringType(), True)])

### Load both the csv files into RDDs

In [ ]:
OrderBookRDD = sc.textFile('/content/OrderBook.csv')
CompanyDetailsRDD = sc.textFile('/content/CompanyDetails.csv')

In [ ]:
OrderBookRDD = OrderBookRDD.map(lambda col:col.split(','))
CompanyDetailsRDD = CompanyDetailsRDD.map(lambda col:col.split(','))

In [ ]:
display(OrderBookRDD.take(2))
CompanyDetailsRDD.take(2)

[['OD10001',
  'C1108',
  'Spring Forks',
  '21103',
  '9487867.48',
  '3-Nov-19',
  '24-Dec-19',
  'Pending'],
 ['OD10002',
  'C1035',
  'Fender',
  '26272',
  '4899369.32',
  '18-Dec-19',
  '14-Mar-20',
  'Pending']]

[['C1001', 'Steel Wheels', 'Pennsylvania', '7172809057', '2007'],
 ['C1002', 'MotoLab Bikes', 'Texas', '6829309477', '2011']]

### Convert both the datasets to DataFrames

In [ ]:
order_df = spark.createDataFrame(OrderBookRDD, schema=order_schema)
company_df = CompanyDetailsRDD.toDF(company_schema)

In [ ]:
order_df = order_df.withColumn('OrderQuantity', order_df['OrderQuantity'].cast('long')).withColumn('CostOfPart', order_df['CostOfPart'].cast('double'))
order_df.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- CompanyID: string (nullable = true)
 |-- partsRequired: string (nullable = true)
 |-- OrderQuantity: long (nullable = true)
 |-- CostOfPart: double (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- ExpectedDeliveryDate: string (nullable = true)
 |-- DeliveryStatus: string (nullable = true)



In [ ]:
company_df = company_df.withColumn('CompanyContact', company_df['CompanyContact'].cast('long')).withColumn('EstablishedYear', company_df['EstablishedYear'].cast('int'))
company_df.printSchema()

root
 |-- CompanyID: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- CompanyLocation: string (nullable = true)
 |-- CompanyContact: long (nullable = true)
 |-- EstablishedYear: integer (nullable = true)



In [ ]:
print('Total records in OrderBook DF : ', order_df.count())
order_df.show(3)

print('Total records in CompanyDetails DF : ', company_df.count())
company_df.show(3)

Total records in OrderBook DF :  1000
+-------+---------+-------------+-------------+----------+---------+--------------------+--------------+
|OrderID|CompanyID|partsRequired|OrderQuantity|CostOfPart|OrderDate|ExpectedDeliveryDate|DeliveryStatus|
+-------+---------+-------------+-------------+----------+---------+--------------------+--------------+
|OD10001|    C1108| Spring Forks|        21103|9487867.48| 3-Nov-19|           24-Dec-19|       Pending|
|OD10002|    C1035|       Fender|        26272|4899369.32|18-Dec-19|           14-Mar-20|       Pending|
|OD10003|    C1174|    Generator|        39789|4008848.49| 7-Dec-19|           31-Jan-20|     Delivered|
+-------+---------+-------------+-------------+----------+---------+--------------------+--------------+
only showing top 3 rows

Total records in CompanyDetails DF :  200
+---------+-------------+---------------+--------------+---------------+
|CompanyID|  CompanyName|CompanyLocation|CompanyContact|EstablishedYear|
+---------+---

### Fetch the details of all the companies that are at least 15 years old

In [ ]:
comp_15yrs_old = company_df.filter((2023 - company_df.EstablishedYear) >= 15)
print('Total numbers of company that are atleast 15 yrs old: ', comp_15yrs_old.count())
comp_15yrs_old.show()

Total numbers of company that are atleast 15 yrs old:  150
+---------+----------------+---------------+--------------+---------------+
|CompanyID|     CompanyName|CompanyLocation|CompanyContact|EstablishedYear|
+---------+----------------+---------------+--------------+---------------+
|    C1001|    Steel Wheels|   Pennsylvania|    7172809057|           2007|
|    C1003|        Lee Auto|     California|    6193601290|           1994|
|    C1004|     Zoom Dynamo|          Texas|    8327309957|           1993|
|    C1005|Quakesons Motors|          Texas|    8175902345|           2002|
|    C1007|     Raido Bikes|         Hawaii|    8086946394|           2001|
|    C1009| Ilmio Jane Moto|   Pennsylvania|    6108768782|           2007|
|    C1011|    Steel Wheels|        Arizona|    6026446720|           1992|
|    C1012|Wingmo Standards|     Washington|    5091629062|           2006|
|    C1013|    Craiko Roads|           Iowa|    3191050692|           1990|
|    C1014|      Reo Bailio| 

### Fetch the name of the company who have ordered “Brakes”

In [ ]:
print('Total numbers of company who ordered "Brakes" : ',join_df.filter("partsRequired=='Brakes'").count())

Total numbers of company who ordered "Brakes" :  42


In [ ]:
join_df = order_df.join(other=company_df, on='CompanyID', how='inner')
join_df.filter("partsRequired=='Brakes'").select(['CompanyName','partsRequired']).show()

+------------------+-------------+
|       CompanyName|partsRequired|
+------------------+-------------+
|       Zoom Dynamo|       Brakes|
|      Craiko Roads|       Brakes|
|        Kioto Moto|       Brakes|
|       Raido Bikes|       Brakes|
|       Raido Bikes|       Brakes|
|      Laser Wheels|       Brakes|
|  Quakesons Motors|       Brakes|
|      Steel Wheels|       Brakes|
|        Reo Bailio|       Brakes|
|        Kioto Moto|       Brakes|
|      Damson Bikes|       Brakes|
|          Lee Auto|       Brakes|
|      Damson Bikes|       Brakes|
|        Kioto Moto|       Brakes|
|Maino World Motors|       Brakes|
|           Neopins|       Brakes|
|      Steel Wheels|       Brakes|
|          Lee Auto|       Brakes|
|  Quakesons Motors|       Brakes|
|           Neopins|       Brakes|
+------------------+-------------+
only showing top 20 rows



### Find the total number of “Exhaust Pipes” that have been ordered from “New York” location

In [ ]:
temp = join_df.filter( (join_df.partsRequired=='Exhaust Pipes') & (join_df.CompanyLocation=='New York') )
print('Total number of “Exhaust Pipes” that have been ordered from “New York” location: ', temp.count())

Total number of “Exhaust Pipes” that have been ordered from “New York” location:  2


### Find the total count of all the orders that has been “Delivered”

In [ ]:
temp = order_df.filter("DeliveryStatus=='Delivered'")
print('Total count of all the orders that has been Delivered: ', temp.count())

Total count of all the orders that has been Delivered:  255


### Find the total count of all the orders that has been “Pending”

In [ ]:
temp = order_df.filter("DeliveryStatus=='Pending'")
print('Total count of all the orders that has been Pending: ', temp.count())

Total count of all the orders that has been Pending:  745


### Find the total quantity and cost of all the parts ordered from “North Carolina” location

In [ ]:
temp = join_df.filter(join_df.CompanyLocation=='North Carolina')

In [ ]:
temp.agg({"OrderQuantity": "sum", "CostOfPart": "sum"}).show()

+--------------------+------------------+
|     sum(CostOfPart)|sum(OrderQuantity)|
+--------------------+------------------+
|1.4936626448999998E8|            974157|
+--------------------+------------------+



### Display the start date, end date and company name who have ordered for “Exhaust Pipes”

In [ ]:
join_df.filter(join_df.partsRequired=='Exhaust Pipes').select(['OrderDate', 'ExpectedDeliveryDate', 'CompanyName']).show()

+---------+--------------------+------------------+
|OrderDate|ExpectedDeliveryDate|       CompanyName|
+---------+--------------------+------------------+
|12-Nov-19|            8-Jan-20|Maino World Motors|
|30-Nov-19|           25-Jan-20|Maino World Motors|
|27-Oct-19|           28-Jan-20|   Ilmio Jane Moto|
|30-Oct-19|            7-Feb-20|       Raido Bikes|
|18-Dec-19|           12-Feb-20|Maino World Motors|
|19-Oct-19|           23-Mar-20|      Laser Wheels|
|14-Nov-19|            9-Feb-20|       Zoom Dynamo|
|24-Oct-19|           22-Mar-20|        Reo Bailio|
| 6-Dec-19|           14-Jan-20|        Reo Bailio|
|20-Oct-19|           26-Feb-20|       Zoom Dynamo|
|27-Nov-19|           27-Feb-20|   Ilmio Jane Moto|
|15-Dec-19|           25-Jan-20|  Wingmo Standards|
|10-Nov-19|           20-Mar-20|   Ilmio Jane Moto|
|18-Oct-19|           13-Mar-20|Maino World Motors|
|25-Oct-19|           28-Feb-20|Maino World Motors|
|12-Oct-19|            2-Mar-20|Maino World Motors|
|30-Oct-19| 